<a href="https://colab.research.google.com/github/Rajat-Kumar-Pandey/MACHINE-LEARNING/blob/main/Drowsiness_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import face_recognition
import cv2
from scipy.spatial import distance as dist
import playsound
from threading import Thread
import numpy as np
import time

# Minimum eye aspect ratio to trigger alarm
Min_Eye_except_ratio = 0.30
# Number of frames the eye aspect ratio must be below threshold to trigger alarm
Eye_AR_Frames = 10

counter = 0
alarm_on = False  # Default alarm state

def sound_alarm(path):
    playsound.playsound(path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def main():
    global counter
    global alarm_on
    Video_capture = cv2.VideoCapture(0)
    Video_capture.set(3, 640)  # Set width
    Video_capture.set(4, 480)  # Set height

    while True:
        ret, frame = Video_capture.read()
        if not ret:
            break

        face_landmarks_list = face_recognition.face_landmarks(frame)
        for face_landmark in face_landmarks_list:
            leftEye = face_landmark['left_eye']
            rightEye = face_landmark['right_eye']
            left_ear = eye_aspect_ratio(leftEye)
            right_ear = eye_aspect_ratio(rightEye)
            ear = (left_ear + right_ear) / 2.0

            # Draw polylines for the eyes
            lpts = np.array(leftEye)
            rpts = np.array(rightEye)
            cv2.polylines(frame, [lpts], True, (255, 255, 0), 1)
            cv2.polylines(frame, [rpts], True, (255, 255, 0), 1)

            # Check if the eye aspect ratio is below the threshold
            if ear < Min_Eye_except_ratio:
                counter += 1
                if counter >= Eye_AR_Frames:
                    if not alarm_on:
                        alarm_on = True  # Start the alarm
                        t = Thread(target=sound_alarm, args=('alarm.wav',))
                        t.daemon = True
                        t.start()
                cv2.putText(frame, "Eye Closed", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            else:
                cv2.putText(frame, "Eye Open", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                counter = 0
                alarm_on = False  # Reset alarm state

            # Display the EAR on the frame
            cv2.putText(frame, f"EAR: {ear:.2f}", (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("Sleep detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    Video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'face_recognition'